In [ ]:
!pip install -q -U google-generativeai

In [ ]:
# Necessary packages
import pathlib
import textwrap
import json

import google.generativeai as genai
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from IPython.display import display
from IPython.display import Markdown
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from google.colab import userdata

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash


In [ ]:
# Muat file Excel
df = pd.read_excel('/content/Sentimen_5.xlsx')

In [ ]:
df.head(2)

,Sentimen,True_Label
0,Saya sangat mndukung sekali,2
1,Saya sangat senang dengan wisata halal karena ...,2


In [ ]:
df['pred_label'] = ''

In [ ]:
df.head(2)

,Sentimen,True_Label,pred_label
0,Saya sangat mndukung sekali,2,
1,Saya sangat senang dengan wisata halal karena ...,2,


In [ ]:
# Convert the DataFrame to JSON using the to_json() method

json_data = df[['Sentimen','pred_label']].to_json(orient='records')

# Print the JSON data
print(json_data)

[{"Sentimen":"Saya sangat mndukung sekali","pred_label":""},{"Sentimen":"Saya sangat senang dengan wisata halal karena dengan sesibuk apapun\/senyaman apapun kita berwisata pasti akan diingatkan untuk selalu taat kepadaNya","pred_label":""},{"Sentimen":"Saya sangat setuju ada pariwisata halal","pred_label":""},{"Sentimen":"saya sangat setuju dan mendukung karena kebijakan pariwisata halal mempunyai potensi besar untuk berkontribusi terhadap pertumbuhan ekonomi wilayah tersebut","pred_label":""},{"Sentimen":"saya sangat setuju dan mendukung karena para wisatawan nyaman beribadah ketika sedang berwisata apalagi dengan fasilitas ibadah yang lengkap dan nyaman","pred_label":""},{"Sentimen":"Saya sangat setuju dan mendukung mengenai kebijakan pariwisata yang halal apalagi mayoritas kami adalah muslim dimana banyak syariat yang harus dijalankan walaupun sedang menjalankan kegiatan ","pred_label":""},{"Sentimen":"Saya sangat setuju dan mendukungkarena akan menjadi alternatif dan kenyamanan te

In [ ]:
prompt = f"""
Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
{json_data}
```
"""

print(prompt)


Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
[{"Sentimen":"Saya sangat mndukung sekali","pred_label":""},{"Sentimen":"Saya sangat senang dengan wisata halal karena dengan sesibuk apapun\/senyaman apapun kita berwisata pasti akan diingatkan untuk selalu taat kepadaNya","pred_label":""},{"Sentimen":"Saya sangat setuju ada pariwisata halal","pred_label":""},{"Sentimen":"saya sangat setuju dan mendukung karena kebijakan pariwisata halal mempunyai potensi besar untuk berkontribusi terhadap p

In [ ]:
response = model.generate_content(prompt)

print(response.text)

```
[{"Sentimen":"Saya sangat mndukung sekali","pred_label":2},{"Sentimen":"Saya sangat senang dengan wisata halal karena dengan sesibuk apapun\/senyaman apapun kita berwisata pasti akan diingatkan untuk selalu taat kepadaNya","pred_label":2},{"Sentimen":"Saya sangat setuju ada pariwisata halal","pred_label":2},{"Sentimen":"saya sangat setuju dan mendukung karena kebijakan pariwisata halal mempunyai potensi besar untuk berkontribusi terhadap pertumbuhan ekonomi wilayah tersebut","pred_label":2},{"Sentimen":"saya sangat setuju dan mendukung karena para wisatawan nyaman beribadah ketika sedang berwisata apalagi dengan fasilitas ibadah yang lengkap dan nyaman","pred_label":2},{"Sentimen":"Saya sangat setuju dan mendukung mengenai kebijakan pariwisata yang halal apalagi mayoritas kami adalah muslim dimana banyak syariat yang harus dijalankan walaupun sedang menjalankan kegiatan ","pred_label":2},{"Sentimen":"Saya sangat setuju dan mendukungkarena akan menjadi alternatif dan kenyamanan ters

In [ ]:
# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Load the cleaned data and convert to DataFrame
data = json.loads(json_data)
df_sample = pd.DataFrame(data)

df_sample

,Sentimen,pred_label
0,Saya sangat mndukung sekali,2
1,Saya sangat senang dengan wisata halal karena ...,2
2,Saya sangat setuju ada pariwisata halal,2
3,saya sangat setuju dan mendukung karena kebija...,2
4,saya sangat setuju dan mendukung karena para w...,2
5,Saya sangat setuju dan mendukung mengenai kebi...,2
6,Saya sangat setuju dan mendukungkarena akan me...,2
7,Saya sangat setuju dengan adanya konsep ini ka...,2
8,saya sangat setuju dengan konsep pariwisata ha...,2
9,Saya sangat setuju karena kebijakan yang diter...,2


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

df['pred_label'] = df_sample['pred_label'].values
df

,Sentimen,True_Label,pred_label
0,Saya sangat mndukung sekali,2,2
1,Saya sangat senang dengan wisata halal karena ...,2,2
2,Saya sangat setuju ada pariwisata halal,2,2
3,saya sangat setuju dan mendukung karena kebija...,2,2
4,saya sangat setuju dan mendukung karena para w...,2,2
5,Saya sangat setuju dan mendukung mengenai kebi...,2,2
6,Saya sangat setuju dan mendukungkarena akan me...,2,2
7,Saya sangat setuju dengan adanya konsep ini ka...,2,2
8,saya sangat setuju dengan konsep pariwisata ha...,2,2
9,Saya sangat setuju karena kebijakan yang diter...,2,2


In [ ]:
# prompt: Menggunakan df DataFrame: saya ingin menyimpan file kedalam bentuk excel

df.to_excel('data 5.xlsx')